# Tutorial 7: Graph Neural Networks

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Finished&color=green)

**Filled notebook:**
[![View on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/GNN_overview.ipynb)
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/GNN_overview.ipynb)  
**Pre-trained models:**
[![View files on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/phlippe/saved_models/tree/main/tutorial7)
[![GoogleDrive](https://img.shields.io/static/v1.svg?logo=google-drive&logoColor=yellow&label=GDrive&message=Download&color=yellow)](https://drive.google.com/drive/folders/1DOTV_oYt5boa-MElbc2izat4VMSc1gob?usp=sharing)   
**Recordings:**
[![YouTube - Part 1](https://img.shields.io/static/v1.svg?logo=youtube&label=YouTube&message=Part%201&color=red)](https://youtu.be/fK7d56Ly9q8)
[![YouTube - Part 2](https://img.shields.io/static/v1.svg?logo=youtube&label=YouTube&message=Part%202&color=red)](https://youtu.be/ZCNSUWe4a_Q)   
**JAX+Flax version:**
[![View on RTD](https://img.shields.io/static/v1.svg?logo=readthedocs&label=RTD&message=View%20On%20RTD&color=8CA1AF)](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/JAX/tutorial7/GNN_overview.html)   
**Author:** Phillip Lippe

<div class="alert alert-info">

**Note:** Interested in JAX? Check out our [JAX+Flax version](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/JAX/tutorial7/GNN_overview.html) of this tutorial!
    
</div>

Trong bài hướng dẫn này, chúng ta sẽ thảo luận về ứng dụng của Neural Networks trên các Graph.

Graph Neural Networks (GNNs) gần đây ngày càng trở nên phổ biến trong cả ứng dụng thực tế lẫn nghiên cứu, bao gồm các lĩnh vực như Social Networks (mạng xã hội), Knowledge Graphs (đồ thị tri thức), Recommender Systems (hệ thống gợi ý) và Bioinformatics (tin sinh học).

Mặc dù lý thuyết và toán học đằng sau GNN thoạt nhìn có vẻ phức tạp, nhưng việc implementation (triển khai code) các model này lại khá đơn giản và hỗ trợ rất tốt cho việc hiểu rõ methodology (phương pháp luận).

Do đó, chúng ta sẽ thảo luận về cách implementation các network layer cơ bản của một GNN, cụ thể là:

Graph Convolutions

Attention Layers

Cuối cùng, chúng ta sẽ áp dụng GNN vào các task (tác vụ) ở các cấp độ:

Node-level (cấp độ nút)

Edge-level (cấp độ cạnh)

Graph-level (cấp độ toàn bộ đồ thị)

Bên dưới, chúng ta sẽ bắt đầu bằng việc import các thư viện tiêu chuẩn. Chúng ta sẽ sử dụng PyTorch Lightning như đã thực hiện trong Tutorial 5 và 6.

In [1]:
## Các thư viện Standard (tiêu chuẩn)
import os
import json
import math
import numpy as np
import time

## Imports cho việc plotting (vẽ biểu đồ)
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # Để export (xuất file ảnh chất lượng cao)
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar (Thanh tiến độ)
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab không cài đặt sẵn PyTorch Lightning theo mặc định. Do đó, chúng ta cài đặt nó ở đây nếu cần thiết
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Đường dẫn đến thư mục nơi các dataset đang có hoặc sẽ được download (ví dụ: CIFAR10)
DATASET_PATH = "../data"
# Đường dẫn đến thư mục nơi các pretrained model được lưu
CHECKPOINT_PATH = "../saved_models/tutorial7"

# Thiết lập seed (để cố định kết quả ngẫu nhiên)
pl.seed_everything(42)

# Đảm bảo rằng tất cả các operation là deterministic trên GPU (nếu được sử dụng) cho mục đích reproducibility (khả năng tái lập kết quả)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

/tmp/ipython-input-615682122.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg', 'pdf') # Để export (xuất file ảnh chất lượng cao)
INFO:lightning_fabric.utilities.seed:Seed set to 42


cpu


chúng ta cũng có một số pre-trained model ở dưới

In [2]:
import urllib.request
from urllib.error import HTTPError
# Github URL nơi các saved models được lưu trữ cho tutorial này
base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial7/"
# Các file cần download
pretrained_files = ["NodeLevelMLP.ckpt", "NodeLevelGNN.ckpt", "GraphLevelGraphConv.ckpt"]

# Tạo checkpoint path nếu nó chưa tồn tại
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# Với mỗi file, kiểm tra xem nó đã tồn tại chưa. Nếu chưa, thử download nó.
for file_name in pretrained_files:
    file_path = os.path.join(CHECKPOINT_PATH, file_name)
    if "/" in file_name:
        os.makedirs(file_path.rsplit("/",1)[0], exist_ok=True)
    if not os.path.isfile(file_path):
        file_url = base_url + file_name
        print(f"Đang download {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
        except HTTPError as e:
            print("Có lỗi xảy ra. Vui lòng thử download file từ GDrive folder, hoặc liên hệ tác giả với full output bao gồm error sau đây:\n", e)

Đang download https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial7/NodeLevelMLP.ckpt...
Đang download https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial7/NodeLevelGNN.ckpt...
Đang download https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial7/GraphLevelGraphConv.ckpt...


## Graph Neural Networks

### Graph representation (Biểu diễn Graph)

Trước khi bắt đầu thảo luận về các hoạt động cụ thể của neural network trên các graph, chúng ta nên xem xét cách để biểu diễn một graph. Về mặt toán học, một graph $\mathcal{G}$ được định nghĩa là một tuple (bộ) bao gồm một set (tập hợp) các nodes/vertices $V$, và một set các edges/links $E$: $\mathcal{G}=(V,E)$. Mỗi edge là một cặp gồm hai vertices, và đại diện cho một kết nối giữa chúng. Ví dụ, hãy nhìn vào graph sau đây:

<center width="100%" style="padding:10px"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/example_graph.svg?raw=1" width="250px"></center>

Các vertices là $V=\{1,2,3,4\}$, và các edges là $E=\{(1,2), (2,3), (2,4), (3,4)\}$. Lưu ý rằng để đơn giản hóa, chúng ta giả định graph là undirected (vô hướng) và do đó không thêm các cặp đối xứng (mirrored pairs) như $(2,1)$. Trong ứng dụng thực tế, các vertices và edge thường có thể có các attributes cụ thể, và các edges thậm chí có thể là directed (có hướng).

Câu hỏi đặt ra là làm thế nào chúng ta có thể biểu diễn sự đa dạng này một cách hiệu quả cho các matrix operations. Thông thường, đối với các edges, chúng ta quyết định giữa hai biến thể: một **adjacency matrix**, hoặc một **list of paired vertex indices**.

**Adjacency matrix** $A$ là một square matrix mà các phần tử của nó chỉ ra liệu các cặp vertices có adjacent (kề nhau), tức là có được connected hay không. Trong trường hợp đơn giản nhất, $A_{ij}$ là 1 nếu có một connection từ node $i$ đến $j$, và ngược lại là 0. Nếu chúng ta có edge attributes hoặc các danh mục edges khác nhau trong một graph, thông tin này cũng có thể được thêm vào matrix. Đối với một undirected graph, hãy nhớ rằng $A$ là một symmetric matrix ($A_{ij}=A_{ji}$). Đối với graph ví dụ ở trên, chúng ta có adjacency matrix sau:

$$
A = \begin{bmatrix}
    0 & 1 & 0 & 0\\
    1 & 0 & 1 & 1\\
    0 & 1 & 0 & 1\\
    0 & 1 & 1 & 0
\end{bmatrix}
$$

Mặc dù việc biểu diễn một graph dưới dạng một list of edges sẽ hiệu quả hơn về mặt memory và (có thể) computation, nhưng việc sử dụng một adjacency matrix lại trực quan hơn và đơn giản hơn để implement. Trong các implementation bên dưới, chúng ta sẽ dựa vào adjacency matrix để giữ cho code đơn giản. Tuy nhiên, các thư viện phổ biến (common libraries) thường sử dụng edge lists, điều mà chúng ta sẽ thảo luận thêm sau này.

Ngoài ra, chúng ta cũng có thể sử dụng list of edges để định nghĩa một **sparse adjacency matrix**, cho phép chúng ta làm việc với nó như thể nó là một **dense matrix**, nhưng cho phép các operations hiệu quả hơn về bộ nhớ. PyTorch hỗ trợ việc này với sub-package `torch.sparse` ([tài liệu](https://pytorch.org/docs/stable/sparse.html)), tuy nhiên nó vẫn đang trong giai đoạn beta-stage (API có thể thay đổi trong tương lai).

### Graph Convolutions

**Graph Convolutional Networks (GCNs)** đã được giới thiệu bởi [Kipf et al.](https://openreview.net/pdf?id=SJU4ayYgl) vào năm 2016 tại Đại học Amsterdam. Ông cũng đã viết một [blog post](https://tkipf.github.io/graph-convolutional-networks/) rất tuyệt về chủ đề này, bài viết được recommended nếu bạn muốn tìm hiểu về GCNs từ một góc nhìn khác. GCNs tương tự như **convolutions** trong xử lý ảnh ở chỗ các tham số "**filter**" thường được **shared** (chia sẻ) trên tất cả các vị trí trong graph. Đồng thời, GCNs dựa vào các phương pháp **message passing**, nghĩa là các vertices trao đổi thông tin với các **neighbors** (hàng xóm), và gửi "**messages**" (thông điệp) cho nhau.

Trước khi xem xét về toán học, chúng ta có thể thử tìm hiểu cách GCNs hoạt động một cách trực quan.
* **Bước 1:** Mỗi node tạo ra một **feature vector** đại diện cho **message** mà nó muốn gửi đến tất cả các neighbors của nó.
* **Bước 2:** Các messages được gửi đến các neighbors, sao cho một node nhận được một message từ mỗi **adjacent node**.

Bên dưới, chúng ta đã trực quan hóa hai bước này cho graph ví dụ của mình.

<center width="100%" style="padding:10px"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/graph_message_passing.svg?raw=1" width="700px"></center>

Nếu chúng ta muốn công thức hóa điều đó bằng các thuật ngữ toán học, trước tiên chúng ta cần quyết định cách kết hợp tất cả các messages mà một node nhận được. Vì số lượng messages thay đổi tùy theo các nodes (do số lượng hàng xóm khác nhau), chúng ta cần một **operation** hoạt động được với bất kỳ số lượng nào. Do đó, cách thông thường là tính tổng (sum) hoặc lấy trung bình (mean).

Với các **features** trước đó của các nodes là $H^{(l)}$, lớp **GCN layer** được định nghĩa như sau:

$$H^{(l+1)} = \sigma\left(\hat{D}^{-1/2}\hat{A}\hat{D}^{-1/2}H^{(l)}W^{(l)}\right)$$

Trong đó:
* $W^{(l)}$ là các tham số **weight** mà chúng ta dùng để biến đổi các **input features** thành các messages ($H^{(l)}W^{(l)}$).
* Đối với **adjacency matrix** $A$, chúng ta cộng thêm **identity matrix** (ma trận đơn vị) để mỗi node cũng tự gửi message của chính nó cho nó: $\hat{A}=A+I$.
* Cuối cùng, để lấy trung bình thay vì tính tổng, chúng ta tính toán **matrix** $\hat{D}$, đây là một **diagonal matrix** (ma trận đường chéo) với $D_{ii}$ biểu thị số lượng neighbors mà node $i$ có.
* $\sigma$ đại diện cho một **activation function** bất kỳ, và không nhất thiết phải là sigmoid (thường thì **ReLU-based activation function** được sử dụng trong GNNs).

Khi **implement** lớp GCN layer trong PyTorch, chúng ta có thể tận dụng các **operations** linh hoạt trên các **tensors**. Thay vì định nghĩa một matrix $\hat{D}$ (việc này tốn kém tính toán), chúng ta có thể chỉ cần chia các messages đã được tính tổng cho số lượng neighbors sau đó. Ngoài ra, chúng ta thay thế weight matrix bằng một **linear layer**, điều này cho phép chúng ta thêm vào một **bias**.

Được viết dưới dạng một **PyTorch module**, lớp GCN layer được định nghĩa như sau:

In [ ]:
class GCNLayer(nn.Module):

    def __init__(self, c_in, c_out):
        super().__init__()
        # Lớp Linear này đóng vai trò là ma trận trọng số W trong công thức
        self.projection = nn.Linear(c_in, c_out)

    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor chứa các node features có shape [batch_size, num_nodes, c_in]
            adj_matrix - Batch các adjacency matrices của graph. Nếu có edge từ i đến j, adj_matrix[b,i,j]=1 ngược lại là 0.
                         Hỗ trợ các directed edges bằng các non-symmetric matrices.
                         Giả định rằng đã thêm các identity connections (tự kết nối với chính nó).
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        # Num neighbours = số lượng incoming edges (cạnh đi vào)
        # Sum theo dimension cuối cùng để đếm xem mỗi node có bao nhiêu hàng xóm kết nối tới nó
        num_neighbours = adj_matrix.sum(dim=-1, keepdims=True)

        # Bước 1: Biến đổi features (tương ứng với H * W trong công thức)
        node_feats = self.projection(node_feats)

        # Bước 2: Message Passing & Aggregation
        # Nhân ma trận kề với node features (bmm = batch matrix multiplication)
        # Thao tác này thực chất là cộng tổng features của các neighbors
        node_feats = torch.bmm(adj_matrix, node_feats)

        # Bước 3: Normalize (Chuẩn hóa)
        # Chia cho số lượng neighbors để lấy trung bình (thay vì chỉ lấy tổng)
        node_feats = node_feats / num_neighbours

        return node_feats

Để hiểu sâu hơn về **GCN layer**, chúng ta có thể áp dụng nó vào **example graph** ở trên. Đầu tiên, hãy xác định một số **node features** và **adjacency matrix** với các **self-connections** đã được thêm vào:

In [ ]:
# Tạo dummy node features (dữ liệu giả lập)
# torch.arange(8) tạo ra các số từ 0 đến 7
# .view(1, 4, 2) định hình lại tensor thành: [Batch size=1, Num nodes=4, Num features=2]
node_feats = torch.arange(8, dtype=torch.float32).view(1, 4, 2)

# Định nghĩa Adjacency Matrix (Ma trận kề)
# Lưu ý: Các phần tử trên đường chéo (diagonal) đều là 1
# Điều này có nghĩa là chúng ta đã thêm self-connections (mỗi node tự kết nối với chính nó)
adj_matrix = torch.Tensor([[[1, 1, 0, 0],
                            [1, 1, 1, 1],
                            [0, 1, 1, 1],
                            [0, 1, 1, 1]]])

print("Node features:\n", node_feats)
print("\nAdjacency matrix:\n", adj_matrix)

Node features:
 tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])

Adjacency matrix:
 tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])


Tiếp theo, hãy áp dụng một **GCN layer** vào nó. Để đơn giản, chúng ta khởi tạo **linear weight matrix** như là một **identity matrix** để các **input features** bằng với các **messages**. Điều này giúp chúng ta dễ dàng xác minh **message passing operation** hơn.

In [ ]:
# Khởi tạo GCNLayer với input channels = 2 và output channels = 2
layer = GCNLayer(c_in=2, c_out=2)

# Gán cứng trọng số (weight) thành Ma trận đơn vị (Identity Matrix)
# [[1., 0.], [0., 1.]] có nghĩa là features sẽ không bị thay đổi khi đi qua Linear layer
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])

# Gán cứng bias thành 0
layer.projection.bias.data = torch.Tensor([0., 0.])

# Thực hiện forward pass mà không tính gradient (vì ta chỉ đang kiểm tra kết quả, không phải training)
with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

Adjacency matrix tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features tensor([[[1., 2.],
         [3., 4.],
         [4., 5.],
         [4., 5.]]])


Như chúng ta có thể thấy, các **output values** của **node** đầu tiên là trung bình cộng của chính nó và **node** thứ hai. Tương tự, chúng ta có thể xác minh cho tất cả các **node** khác.

Tuy nhiên, trong một **GNN**, chúng ta cũng muốn cho phép việc **feature exchange** (trao đổi đặc trưng) giữa các **nodes** nằm ngoài phạm vi **neighbors** (hàng xóm) trực tiếp của nó. Điều này có thể đạt được bằng cách áp dụng nhiều **GCN layers**, tạo nên **final layout** (bố cục cuối cùng) của một **GNN**. Một **GNN** có thể được xây dựng bởi một chuỗi các **GCN layers** và các **non-linearities** (hàm phi tuyến) chẳng hạn như **ReLU**. Để hình dung, hãy xem bên dưới (nguồn ảnh - [Thomas Kipf, 2016](https://tkipf.github.io/graph-convolutional-networks/)).

<center width="100%" style="padding: 10px"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/gcn_network.png?raw=1" width="600px"></center>

Tuy nhiên, một vấn đề mà chúng ta có thể thấy từ việc nhìn vào ví dụ trên là các **output features** cho **nodes** 3 và 4 giống hệt nhau bởi vì chúng có cùng các **adjacent nodes** (bao gồm cả chính nó). Do đó, các **GCN layers** có thể làm cho mạng quên đi các **node-specific information** (thông tin đặc thù của node) nếu chúng ta chỉ lấy trung bình trên tất cả các **messages**.

Nhiều cải tiến khả thi đã được đề xuất. Trong khi lựa chọn đơn giản nhất có thể là sử dụng **residual connections**, cách tiếp cận phổ biến hơn là hoặc đặt trọng số cho các **self-connections** cao hơn, hoặc định nghĩa một **weight matrix** riêng biệt cho các **self-connections**.

Ngoài ra, chúng ta có thể xem lại một khái niệm từ bài hướng dẫn trước: **attention**.

### Graph Attention

Nếu bạn còn nhớ bài hướng dẫn trước, **attention** mô tả một **weighted average** (trung bình có trọng số) của nhiều phần tử, với các trọng số được tính toán động (dynamically computed) dựa trên một **input query** và các **keys** của phần tử (nếu bạn chưa đọc Tutorial 6, bạn nên xem qua ít nhất phần đầu tiên có tên [What is Attention?](https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial6/Transformers_and_MHAttention.html#What-is-Attention?)).

Khái niệm này có thể được áp dụng tương tự cho các **graphs**, một trong số đó là **Graph Attention Network** (được gọi là **GAT**, đề xuất bởi [Velickovic et al., 2017](https://arxiv.org/abs/1710.10903)).

Tương tự như **GCN**, lớp **graph attention layer** tạo ra một **message** cho mỗi **node** sử dụng một **linear layer/weight matrix**. Đối với phần **attention**, nó sử dụng **message** từ chính **node** đó đóng vai trò là **query**, và các **messages** cần tính trung bình đóng vai trò là cả **keys** và **values** (lưu ý rằng điều này bao gồm cả **message** gửi cho chính nó).

Hàm tính điểm (Score function) $f_{attn}$ được **implemented** như là một **one-layer MLP** giúp ánh xạ **query** và **key** thành một giá trị đơn lẻ. **MLP** trông như sau (nguồn ảnh - [Velickovic et al.](https://arxiv.org/abs/1710.10903)):

<center width="100%" style="padding:10px"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/graph_attention_MLP.svg?raw=1" width="250px"></center>

Trong đó:
* $h_i$ và $h_j$ là các **features** gốc từ **node** $i$ và $j$, và đại diện cho các **messages** của **layer** với $\mathbf{W}$ là **weight matrix**.
* $\mathbf{a}$ là **weight matrix** của **MLP**, có **shape** $[1,2\times d_{\text{message}}]$.
* $\alpha_{ij}$ là **attention weight** cuối cùng từ **node** $i$ đến $j$.

Việc tính toán có thể được mô tả như sau:

$$\alpha_{ij} = \frac{\exp\left(\text{LeakyReLU}\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_j\right]\right)\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\text{LeakyReLU}\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_k\right]\right)\right)}$$

Toán tử $||$ đại diện cho sự **concatenation** (phép nối), và $\mathcal{N}_i$ là các chỉ số (indices) của các **neighbors** của **node** $i$.

Lưu ý rằng trái ngược với thực hành thông thường, chúng ta áp dụng một **non-linearity** (ở đây là **LeakyReLU**) *trước* khi **softmax** trên các phần tử. Mặc dù thoạt đầu nó có vẻ như một thay đổi nhỏ, nhưng nó rất quan trọng để **attention** phụ thuộc vào **input** gốc. Cụ thể, hãy thử loại bỏ **non-linearity** trong một giây, và thử đơn giản hóa biểu thức:

$$
\begin{split}
    \alpha_{ij} & = \frac{\exp\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_j\right]\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}\left[\mathbf{W}h_i||\mathbf{W}h_k\right]\right)}\\[5pt]
    & = \frac{\exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i+\mathbf{a}_{:,d/2:}\mathbf{W}h_j\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i+\mathbf{a}_{:,d/2:}\mathbf{W}h_k\right)}\\[5pt]
    & = \frac{\exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i\right)\cdot\exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_j\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}_{:,:d/2}\mathbf{W}h_i\right)\cdot\exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_k\right)}\\[5pt]
    & = \frac{\exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_j\right)}{\sum_{k\in\mathcal{N}_i} \exp\left(\mathbf{a}_{:,d/2:}\mathbf{W}h_k\right)}\\
\end{split}
$$

Chúng ta có thể thấy rằng nếu không có **non-linearity**, thành phần **attention** với $h_i$ thực sự tự triệt tiêu lẫn nhau, dẫn đến việc **attention** trở nên độc lập với chính **node** đó (node $i$). Do đó, chúng ta sẽ gặp phải vấn đề tương tự như **GCN** là tạo ra các **output features** giống nhau cho các **nodes** có cùng **neighbors**. Đây là lý do tại sao **LeakyReLU** là cốt yếu và giúp thêm sự phụ thuộc vào $h_i$ cho **attention**.

Một khi chúng ta có được tất cả các **attention factors**, chúng ta có thể tính toán **output features** cho mỗi **node** bằng cách thực hiện **weighted average**:

$$h_i'=\sigma\left(\sum_{j\in\mathcal{N}_i}\alpha_{ij}\mathbf{W}h_j\right)$$

$\sigma$ lại là một **non-linearity** khác, giống như trong **GCN layer**. Về mặt hình ảnh, chúng ta có thể biểu diễn toàn bộ quá trình **message passing** trong một **attention layer** như sau (nguồn ảnh - [Velickovic et al.](https://arxiv.org/abs/1710.10903)):

<center width="100%"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/graph_attention.jpeg?raw=1" width="400px"></center>

Để tăng tính **expressiveness** (khả năng biểu diễn) của **graph attention network**, [Velickovic et al.](https://arxiv.org/abs/1710.10903) đã đề xuất mở rộng nó thành nhiều **heads** (đầu) tương tự như khối **Multi-Head Attention** trong **Transformers**. Điều này dẫn đến việc $N$ **attention layers** được áp dụng song song (**in parallel**). Trong hình ảnh trên, nó được trực quan hóa bằng ba màu mũi tên khác nhau (xanh lá, xanh dương và tím) mà sau đó được nối lại (**concatenated**). Việc lấy trung bình chỉ được áp dụng cho lớp **prediction layer** cuối cùng trong mạng.

Sau khi đã thảo luận chi tiết về **graph attention layer**, chúng ta có thể **implement** nó bên dưới:

In [ ]:
class GATLayer(nn.Module):

    def __init__(self, c_in, c_out, num_heads=1, concat_heads=True, alpha=0.2):
        """
        Inputs:
            c_in - Dimensionality (số chiều) của input features
            c_out - Dimensionality (số chiều) của output features
            num_heads - Số lượng heads, tức là các cơ chế attention được áp dụng song song (in parallel).
                        Các output features được chia đều cho các heads nếu concat_heads=True.
            concat_heads - Nếu True, output của các heads khác nhau sẽ được nối lại (concatenated) thay vì lấy trung bình (averaged).
            alpha - Negative slope của LeakyReLU activation.
        """
        super().__init__()
        self.num_heads = num_heads
        self.concat_heads = concat_heads
        if self.concat_heads:
            assert c_out % num_heads == 0, "Số lượng output features phải là bội số của số lượng heads."
            c_out = c_out // num_heads

        # Các sub-modules và tham số cần thiết trong layer
        self.projection = nn.Linear(c_in, c_out * num_heads)
        self.a = nn.Parameter(torch.Tensor(num_heads, 2 * c_out)) # Mỗi head có một vector trọng số a riêng
        self.leakyrelu = nn.LeakyReLU(alpha)

        # Initialization (Khởi tạo) từ bản implementation gốc
        nn.init.xavier_uniform_(self.projection.weight.data, gain=1.414)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

    def forward(self, node_feats, adj_matrix, print_attn_probs=False):
        """
        Inputs:
            node_feats - Input features của node. Shape: [batch_size, c_in]
            adj_matrix - Adjacency matrix bao gồm cả self-connections. Shape: [batch_size, num_nodes, num_nodes]
            print_attn_probs - Nếu True, các attention weights sẽ được in ra trong quá trình forward pass (để debug)
        """
        batch_size, num_nodes = node_feats.size(0), node_feats.size(1)

        # Áp dụng linear layer và sắp xếp nodes theo head
        node_feats = self.projection(node_feats)
        node_feats = node_feats.view(batch_size, num_nodes, self.num_heads, -1)

        # Chúng ta cần tính toán attention logits cho mọi edge trong adjacency matrix
        # Làm việc này trên tất cả các tổ hợp nodes có thể (N x N) rất tốn kém
        # => Tạo một tensor [W*h_i || W*h_j] với i và j là các chỉ số (indices) của tất cả các edges
        edges = adj_matrix.nonzero(as_tuple=False) # Trả về các chỉ số nơi adjacency matrix khác 0 => edges
        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, -1)

        # Tính toán chỉ số phẳng (flat index) để lấy feature của node nguồn và node đích
        edge_indices_row = edges[:,0] * num_nodes + edges[:,1]
        edge_indices_col = edges[:,0] * num_nodes + edges[:,2]

        # Nối (Concatenate) features của cặp node (i, j) lại với nhau
        a_input = torch.cat([
            torch.index_select(input=node_feats_flat, index=edge_indices_row, dim=0),
            torch.index_select(input=node_feats_flat, index=edge_indices_col, dim=0)
        ], dim=-1) # Index select trả về một tensor với node_feats_flat được đánh chỉ mục tại các vị trí mong muốn dọc theo dim=0

        # Tính toán output của attention MLP (độc lập cho mỗi head)
        # einsum: thực hiện phép nhân ma trận hiệu quả
        attn_logits = torch.einsum('bhc,hc->bh', a_input, self.a)
        attn_logits = self.leakyrelu(attn_logits)

        # Map danh sách các giá trị attention ngược lại vào một matrix
        # Khởi tạo matrix với giá trị cực nhỏ (-9e15) để khi qua Softmax nó sẽ xấp xỉ 0 (masking)
        attn_matrix = attn_logits.new_zeros(adj_matrix.shape+(self.num_heads,)).fill_(-9e15)
        attn_matrix[adj_matrix[...,None].repeat(1,1,1,self.num_heads) == 1] = attn_logits.reshape(-1)

        # Weighted average của attention (Tính Softmax)
        attn_probs = F.softmax(attn_matrix, dim=2)
        if print_attn_probs:
            print("Attention probs\n", attn_probs.permute(0, 3, 1, 2))

        # Tổng hợp features theo trọng số attention
        node_feats = torch.einsum('bijh,bjhc->bihc', attn_probs, node_feats)

        # Nếu concat heads, chúng ta thực hiện reshape. Ngược lại, lấy trung bình (mean)
        if self.concat_heads:
            node_feats = node_feats.reshape(batch_size, num_nodes, -1)
        else:
            node_feats = node_feats.mean(dim=2)

        return node_feats

Một lần nữa, chúng ta có thể áp dụng **graph attention layer** vào **example graph** ở trên của chúng ta để hiểu rõ hơn về các cơ chế hoạt động (**dynamics**).

Như trước đây, **input layer** được khởi tạo là một **identity matrix** (ma trận đơn vị), nhưng chúng ta thiết lập $\mathbf{a}$ là một vector chứa các số bất kỳ để thu được các giá trị **attention** khác nhau. Chúng ta sử dụng hai **heads** để minh họa các cơ chế **attention** song song (**parallel**) và độc lập đang hoạt động trong **layer**.

In [ ]:
# Khởi tạo GATLayer
# c_in=2, c_out=2, num_heads=2
# Vì concat_heads=True (mặc định), mỗi head sẽ phụ trách output ra 1 feature (2 feature tổng / 2 heads)
layer = GATLayer(2, 2, num_heads=2)

# Gán trọng số projection là Ma trận đơn vị (Identity Matrix)
# Điều này giúp features không bị biến đổi giá trị khi đi qua Linear layer (để dễ kiểm tra tính toán)
layer.projection.weight.data = torch.Tensor([[1., 0.], [0., 1.]])
layer.projection.bias.data = torch.Tensor([0., 0.])

# Gán trọng số a (attention mechanism parameters) thủ công
# Chúng ta có 2 heads, nên tensor có shape tương ứng
# Head 1: [-0.2, 0.3] -> Sẽ có cách "chú ý" riêng
# Head 2: [0.1, -0.1] -> Sẽ có cách "chú ý" khác hẳn Head 1
layer.a.data = torch.Tensor([[-0.2, 0.3], [0.1, -0.1]])

# Thực hiện forward pass không tính gradient
with torch.no_grad():
    # print_attn_probs=True sẽ kích hoạt dòng lệnh in ra ma trận attention trong hàm forward
    out_feats = layer(node_feats, adj_matrix, print_attn_probs=True)

print("Adjacency matrix", adj_matrix)
print("Input features", node_feats)
print("Output features", out_feats)

Attention probs
 tensor([[[[0.3543, 0.6457, 0.0000, 0.0000],
          [0.1096, 0.1450, 0.2642, 0.4813],
          [0.0000, 0.1858, 0.2885, 0.5257],
          [0.0000, 0.2391, 0.2696, 0.4913]],

         [[0.5100, 0.4900, 0.0000, 0.0000],
          [0.2975, 0.2436, 0.2340, 0.2249],
          [0.0000, 0.3838, 0.3142, 0.3019],
          [0.0000, 0.4018, 0.3289, 0.2693]]]])
Adjacency matrix tensor([[[1., 1., 0., 0.],
         [1., 1., 1., 1.],
         [0., 1., 1., 1.],
         [0., 1., 1., 1.]]])
Input features tensor([[[0., 1.],
         [2., 3.],
         [4., 5.],
         [6., 7.]]])
Output features tensor([[[1.2913, 1.9800],
         [4.2344, 3.7725],
         [4.6798, 4.8362],
         [4.5043, 4.7351]]])


Chúng tôi khuyến khích bạn thử tự tính toán **attention matrix** cho ít nhất một **head** và một **node**. Các giá trị sẽ là 0 tại những nơi không tồn tại **edge** giữa $i$ và $j$.

Đối với các vị trí khác, chúng ta thấy một tập hợp đa dạng các **attention probabilities**. Hơn nữa, các **output features** của **node** 3 và 4 bây giờ đã khác nhau mặc dù chúng có cùng các **neighbors**.

## PyTorch Geometric

Chúng ta đã đề cập trước đó rằng việc **implementing graph networks** với **adjacency matrix** thì đơn giản và trực quan nhưng có thể **computationally expensive** (tốn kém về mặt tính toán) cho các **large graphs**.

Nhiều **real-world graphs** có thể đạt tới hơn 200k **nodes**, khiến cho các **implementations** dựa trên **adjacency matrix** bị thất bại. Có rất nhiều **optimizations** khả thi khi **implementing GNNs**, và may mắn thay, tồn tại các **packages** cung cấp các **layers** như vậy.

Các **packages** phổ biến nhất cho **PyTorch** là [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/) và [Deep Graph Library](https://www.dgl.ai/) (cái sau thực ra là **framework agnostic**). Việc sử dụng cái nào phụ thuộc vào **project** bạn đang lên kế hoạch làm và sở thích cá nhân.

Trong **tutorial** này, chúng ta sẽ xem xét **PyTorch Geometric** như là một phần của gia đình **PyTorch**. Tương tự như **PyTorch Lightning**, **PyTorch Geometric** không được cài đặt mặc định trên **Google Colab** (và thực ra cũng không có trong môi trường `dl2021` của chúng ta do có quá nhiều **dependencies** không cần thiết cho các bài thực hành). Do đó, hãy **import** và/hoặc **install** nó bên dưới:

In [ ]:
# torch geometric
try:
    import torch_geometric
except ModuleNotFoundError:
    # Cài đặt các packages torch geometric với CUDA+PyTorch version cụ thể.
    # Xem https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html để biết thêm chi tiết
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    # Cài đặt các thư viện phụ thuộc (dependencies) quan trọng
    # -f chỉ định đường dẫn tìm file cài đặt (wheels) tương thích
    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric

    import torch_geometric

# Import các module con thường dùng
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data

RDKit WARNING: [19:12:50] Enabling RDKit 2019.09.3 jupyter extensions


**PyTorch Geometric** cung cấp cho chúng ta một tập hợp các **graph layers** phổ biến, bao gồm lớp **GCN** và **GAT** mà chúng ta đã **implemented** ở trên.

Ngoài ra, tương tự như **torchvision** của PyTorch, nó cung cấp các **common graph datasets** và các **transformations** trên chúng để đơn giản hóa quá trình **training**. So với **implementation** của chúng ta ở trên, **PyTorch Geometric** sử dụng một danh sách các **index pairs** (cặp chỉ số) để biểu diễn các **edges**. Chi tiết về thư viện này sẽ được khám phá sâu hơn trong các **experiments** của chúng ta.

Trong các **tasks** bên dưới, chúng ta muốn cho phép mình chọn từ vô số các **graph layers**. Vì vậy, chúng ta định nghĩa lại bên dưới một **dictionary** để truy cập chúng bằng một **string**:

In [ ]:
gnn_layer_by_name = {
    "GCN": geom_nn.GCNConv,
    "GAT": geom_nn.GATConv,
    "GraphConv": geom_nn.GraphConv
}

Ngoài **GCN** và **GAT**, chúng ta đã thêm vào lớp `geom_nn.GraphConv` ([tài liệu](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GraphConv)).

**GraphConv** là một **GCN** với một **weight matrix** riêng biệt cho các **self-connections**. Về mặt toán học, điều này sẽ là:

$$
\mathbf{x}_i^{(l+1)} = \mathbf{W}^{(l + 1)}_1 \mathbf{x}_i^{(l)} + \mathbf{W}^{(\ell + 1)}_2 \sum_{j \in \mathcal{N}_i} \mathbf{x}_j^{(l)}
$$

Trong công thức này, các **messages** của **neighbor** được cộng lại (**added**) thay vì lấy trung bình (**averaged**). Tuy nhiên, **PyTorch Geometric** cung cấp tham số `aggr` để chuyển đổi giữa **summing**, **averaging**, và **max pooling**.

## Experiments on graph structures (Thực nghiệm trên cấu trúc đồ thị)

Các tác vụ trên dữ liệu có cấu trúc graph (**graph-structured data**) có thể được chia thành ba nhóm: **node-level**, **edge-level** và **graph-level**.

Các cấp độ khác nhau này mô tả việc chúng ta muốn thực hiện **classification/regression** ở cấp độ nào. Chúng ta sẽ thảo luận chi tiết hơn về cả ba loại này ở bên dưới.

### Node-level tasks: Semi-supervised node classification

Các **Node-level tasks** có mục tiêu là phân loại các **nodes** trong một **graph**.

Thông thường, chúng ta được cung cấp một **single, large graph** (một đồ thị đơn lớn) với hơn 1000 **nodes**, trong đó một lượng nhất định các **nodes** đã được gán nhãn (**labeled**). Chúng ta học cách phân loại các ví dụ **labeled** đó trong quá trình **training** và cố gắng **generalize** (khái quát hóa) sang các **unlabeled nodes** (các node chưa có nhãn).

Một ví dụ phổ biến mà chúng ta sẽ sử dụng trong **tutorial** này là **Cora dataset**, một mạng lưới trích dẫn (**citation network**) giữa các bài báo khoa học.

[Image of citation network graph visualization]

**Cora** bao gồm 2708 ấn phẩm khoa học với các liên kết giữa chúng đại diện cho việc trích dẫn của bài báo này đối với bài báo khác. **Task** ở đây là phân loại mỗi ấn phẩm vào một trong bảy **classes**.

Mỗi ấn phẩm được đại diện bởi một **bag-of-words vector**. Điều này có nghĩa là chúng ta có một **vector** gồm 1433 phần tử cho mỗi ấn phẩm, trong đó số 1 tại **feature** $i$ chỉ ra rằng từ thứ $i$ của một từ điển xác định trước có xuất hiện trong bài báo đó.

Các biểu diễn **Binary bag-of-words** thường được sử dụng khi chúng ta cần các **encodings** rất đơn giản, và đã có sẵn trực giác về những từ ngữ nào sẽ xuất hiện trong một mạng lưới. Có nhiều phương pháp tiếp cận tốt hơn nhiều, nhưng chúng ta sẽ để dành phần này cho các khóa học **NLP** thảo luận.

Chúng ta sẽ tải **dataset** bên dưới:

In [ ]:
cora_dataset = torch_geometric.datasets.Planetoid(root=DATASET_PATH, name="Cora")

Hãy xem xét cách **PyTorch Geometric** biểu diễn **graph data**.

Lưu ý rằng mặc dù chúng ta có một **single graph** (đồ thị đơn), **PyTorch Geometric** vẫn trả về một **dataset** để đảm bảo tính tương thích (**compatibility**) với các **datasets** khác.

In [ ]:
cora_dataset[0]

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

Giải thích chi tiết các thuộc tính:
x=[2708, 1433] (Node Features)

Ý nghĩa: Ma trận đặc trưng của các node.

2708: Số lượng nodes (tương ứng với 2708 bài báo khoa học trong dataset).

1433: Số lượng features của mỗi node (kích thước vector Bag-of-words). Mỗi bài báo được mô tả bởi sự xuất hiện của 1433 từ khóa.

edge_index=[2, 10556] (Graph Connectivity)

Ý nghĩa: Cấu trúc liên kết của đồ thị (các cạnh).

10556: Tổng số lượng edges (các lượt trích dẫn) trong đồ thị.

2: Tại sao lại là 2? PyTorch Geometric lưu trữ cạnh dưới dạng COO format (Coordinate Format).

Hàng 1: Index của node nguồn (Source nodes).

Hàng 2: Index của node đích (Target nodes).

Ví dụ: Một cột [0, 5] nghĩa là có cạnh nối từ Node 0 đến Node 5.

y=[2708] (Labels)

Ý nghĩa: Nhãn phân loại của từng node (Ground truth).

2708: Mỗi bài báo sẽ có 1 nhãn tương ứng (từ 0 đến 6, đại diện cho 7 chủ đề khoa học khác nhau).

train_mask=[2708], val_mask=[2708], test_mask=[2708]

Ý nghĩa: Các mặt nạ (Masks) dùng để chia dữ liệu.

Đây là các mảng Boolean (True hoặc False).

Vì chúng ta chỉ có 1 Graph duy nhất, chúng ta không thể chia graph ra làm 3 phần rời nhau (sẽ mất kết nối). Thay vào đó, ta dùng mask để đánh dấu:

train_mask: Node nào được dùng để tính loss và cập nhật trọng số (Training).

val_mask: Node nào dùng để tinh chỉnh mô hình (Validation).

test_mask: Node nào dùng để đánh giá cuối cùng (Testing).

Kích thước 2708 nghĩa là mỗi node đều có một trạng thái True/False tương ứng trong từng mask.

**Graph** được biểu diễn bởi một `Data` object ([tài liệu](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data)) mà chúng ta có thể truy cập như một **standard Python namespace**.

**Edge index tensor** là danh sách các **edges** trong **graph** và chứa phiên bản đối xứng (**mirrored version**) của mỗi **edge** đối với các **undirected graphs**.

Các `train_mask`, `val_mask`, và `test_mask` là các **boolean masks** chỉ ra những **nodes** nào chúng ta nên sử dụng cho **training**, **validation**, và **testing**.

`x` tensor là **feature tensor** của 2708 ấn phẩm (**publications**) của chúng ta, và `y` là các **labels** cho tất cả các **nodes**.

Sau khi đã xem qua dữ liệu, chúng ta có thể **implement** một **graph neural network** đơn giản. **GNN** áp dụng một chuỗi các **graph layers** (GCN, GAT, hoặc GraphConv), **ReLU** làm **activation function**, và **dropout** để **regularization**. Xem bên dưới để biết **implementation** cụ thể.

In [ ]:
class GNNModel(nn.Module):

    def __init__(self, c_in, c_hidden, c_out, num_layers=2, layer_name="GCN", dp_rate=0.1, **kwargs):
        """
        Inputs:
            c_in - Dimension của input features
            c_hidden - Dimension của hidden features
            c_out - Dimension của output features. Thường là số lượng classes trong classification
            num_layers - Số lượng các graph layers "ẩn" (hidden)
            layer_name - String tên của graph layer cần sử dụng
            dp_rate - Dropout rate được áp dụng xuyên suốt network
            kwargs - Các tham số bổ sung cho graph layer (ví dụ: số lượng heads cho GAT)
        """
        super().__init__()
        # Lấy lớp layer tương ứng từ dictionary (đã định nghĩa trước đó) dựa trên tên
        gnn_layer = gnn_layer_by_name[layer_name]

        layers = []
        in_channels, out_channels = c_in, c_hidden

        # Vòng lặp tạo các lớp ẩn (Hidden Layers)
        for l_idx in range(num_layers-1):
            layers += [
                gnn_layer(in_channels=in_channels,
                          out_channels=out_channels,
                          **kwargs),
                nn.ReLU(inplace=True),
                nn.Dropout(dp_rate)
            ]
            in_channels = c_hidden # Cập nhật input channels cho lớp tiếp theo

        # Thêm lớp output cuối cùng (không có ReLU/Dropout sau lớp này để lấy logits)
        layers += [gnn_layer(in_channels=in_channels,
                             out_channels=c_out,
                             **kwargs)]

        # Đóng gói danh sách layers vào nn.ModuleList để PyTorch quản lý tham số
        self.layers = nn.ModuleList(layers)

    def forward(self, x, edge_index):
        """
        Inputs:
            x - Input features cho mỗi node
            edge_index - List các cặp chỉ số vertex biểu diễn các edges trong graph (ký hiệu của PyTorch Geometric)
        """
        for l in self.layers:
            # Đối với các graph layers, chúng ta cần thêm tensor "edge_index" như là input bổ sung
            # Tất cả các PyTorch Geometric graph layer đều thừa kế class "MessagePassing", do đó
            # chúng ta có thể đơn giản là kiểm tra loại class (class type).
            if isinstance(l, geom_nn.MessagePassing):
                x = l(x, edge_index)
            else:
                # Các lớp như ReLU hay Dropout chỉ cần input x
                x = l(x)
        return x

Một thực hành tốt trong các **node-level tasks** là tạo ra một **MLP baseline** được áp dụng cho mỗi **node** một cách độc lập.

Theo cách này, chúng ta có thể xác minh xem liệu việc thêm **graph information** vào **model** thực sự có cải thiện **prediction** hay không, hay là không cần thiết.

Cũng có thể xảy ra trường hợp là các **features** trên mỗi **node** đã đủ **expressive** (có tính biểu đạt) để chỉ rõ về một **class** cụ thể mà không cần quan tâm đến hàng xóm. Để kiểm tra điều này, chúng ta **implement** một **MLP** đơn giản bên dưới.

In [ ]:
class MLPModel(nn.Module):

    def __init__(self, c_in, c_hidden, c_out, num_layers=2, dp_rate=0.1):
        """
        Inputs:
            c_in - Dimension của input features
            c_hidden - Dimension của hidden features
            c_out - Dimension của output features. Thường là số lượng classes trong classification
            num_layers - Số lượng hidden layers
            dp_rate - Dropout rate được áp dụng xuyên suốt network
        """
        super().__init__()
        layers = []
        in_channels, out_channels = c_in, c_hidden

        # Vòng lặp tạo các lớp ẩn (Hidden Layers)
        for l_idx in range(num_layers-1):
            layers += [
                nn.Linear(in_channels, out_channels),
                nn.ReLU(inplace=True),
                nn.Dropout(dp_rate)
            ]
            in_channels = c_hidden

        # Thêm lớp Linear cuối cùng để ra output
        layers += [nn.Linear(in_channels, c_out)]

        # Sử dụng nn.Sequential vì luồng dữ liệu là tuần tự, không cần xử lý edge_index phức tạp
        self.layers = nn.Sequential(*layers)

    def forward(self, x, *args, **kwargs):
        """
        Inputs:
            x - Input features cho mỗi node
        """
        # *args, **kwargs được thêm vào để tương thích với cấu trúc gọi hàm của GNN
        # GNN sẽ truyền vào (x, edge_index), nhưng MLP chỉ cần x và lờ đi edge_index
        return self.layers(x)

Cuối cùng, chúng ta có thể hợp nhất các **models** vào một **PyTorch Lightning module**, module này sẽ xử lý việc **training**, **validation**, và **testing** cho chúng ta.

In [ ]:
class NodeLevelGNN(pl.LightningModule):

    def __init__(self, model_name, **model_kwargs):
        super().__init__()
        # Lưu các hyperparameters (tham số siêu hình) để tiện cho việc checkpoint và log
        self.save_hyperparameters()

        # Chọn kiến trúc model dựa trên tên
        if model_name == "MLP":
            self.model = MLPModel(**model_kwargs)
        else:
            self.model = GNNModel(**model_kwargs)

        # Hàm loss function cho bài toán phân loại nhiều lớp (Multi-class classification)
        self.loss_module = nn.CrossEntropyLoss()

    def forward(self, data, mode="train"):
        x, edge_index = data.x, data.edge_index

        # Forward pass qua model (GNN hoặc MLP)
        x = self.model(x, edge_index)

        # Chỉ tính toán loss trên các nodes tương ứng với mask
        # Vì đây là Semi-supervised learning trên 1 graph duy nhất
        if mode == "train":
            mask = data.train_mask
        elif mode == "val":
            mask = data.val_mask
        elif mode == "test":
            mask = data.test_mask
        else:
            assert False, f"Unknown forward mode: {mode}"

        # Tính Loss chỉ trên các node được phép nhìn thấy (theo mask)
        loss = self.loss_module(x[mask], data.y[mask])

        # Tính Accuracy
        acc = (x[mask].argmax(dim=-1) == data.y[mask]).sum().float() / mask.sum()
        return loss, acc

    def configure_optimizers(self):
        # Chúng ta dùng SGD ở đây, nhưng Adam cũng hoạt động tốt
        # Weight decay giúp giảm overfitting
        optimizer = optim.SGD(self.parameters(), lr=0.1, momentum=0.9, weight_decay=2e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        # Log kết quả để theo dõi
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)

Ngoài **Lightning module**, chúng ta định nghĩa một **training function** bên dưới.

Vì chúng ta có một **single graph** (đồ thị đơn), chúng ta sử dụng **batch size** là 1 cho **data loader** và chia sẻ cùng một **data loader** cho **train**, **validation**, và **test set** (**mask** được chọn bên trong **Lightning module**).

Bên cạnh đó, chúng ta thiết lập tham số `enable_progress_bar` thành False vì nó thường hiển thị **progress** (tiến độ) trên mỗi **epoch**, nhưng một **epoch** ở đây chỉ bao gồm một **single step**. Phần còn lại của code rất giống với những gì chúng ta đã thấy trong Tutorial 5 và 6.

In [ ]:
def train_node_classifier(model_name, dataset, **model_kwargs):
    pl.seed_everything(42)
    # Tạo DataLoader với batch_size=1 vì chúng ta chỉ có 1 Graph duy nhất
    node_data_loader = geom_data.DataLoader(dataset, batch_size=1)

    # Tạo một PyTorch Lightning trainer với callback để lưu model
    root_dir = os.path.join(CHECKPOINT_PATH, "NodeLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)

    trainer = pl.Trainer(default_root_dir=root_dir,
                         # ModelCheckpoint: Tự động lưu lại trọng số của model tốt nhất dựa trên val_acc
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         # Tự động chọn GPU nếu có, ngược lại dùng CPU
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=200,
                         enable_progress_bar=False) # False vì epoch size là 1 (quá nhanh để hiển thị thanh tiến độ)

    trainer.logger._default_hp_metric = None # Tham số logging tùy chọn mà chúng ta không cần

    # Kiểm tra xem pretrained model đã tồn tại chưa. Nếu có, load nó và bỏ qua training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"NodeLevel{model_name}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Tìm thấy pretrained model, đang loading...")
        model = NodeLevelGNN.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything()
        # Khởi tạo model mới: NodeLevelGNN
        model = NodeLevelGNN(model_name=model_name, c_in=dataset.num_node_features, c_out=dataset.num_classes, **model_kwargs)
        # Bắt đầu huấn luyện (dùng validation set trùng với train set trong loader, nhưng logic mask sẽ xử lý việc tách biệt)
        trainer.fit(model, node_data_loader, node_data_loader)
        # Load lại model tốt nhất sau khi train xong
        model = NodeLevelGNN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

    # Test model tốt nhất trên test set
    test_result = trainer.test(model, node_data_loader, verbose=False)

    # Tính toán thêm độ chính xác trên tập Train và Val để báo cáo đầy đủ
    batch = next(iter(node_data_loader))
    batch = batch.to(model.device)
    _, train_acc = model.forward(batch, mode="train")
    _, val_acc = model.forward(batch, mode="val")

    result = {"train": train_acc,
              "val": val_acc,
              "test": test_result[0]['test_acc']}
    return model, result

Cuối cùng, chúng ta có thể huấn luyện các mô hình của mình. Trước tiên, hãy huấn luyện MLP đơn giản:

In [ ]:
# Hàm nhỏ để in ra các điểm số kiểm tra (test scores)
def print_results(result_dict):
    if "train" in result_dict:
        # In ra độ chính xác tập Train, định dạng 2 chữ số thập phân
        print(f"Train accuracy: {(100.0*result_dict['train']):4.2f}%")
    if "val" in result_dict:
        # In ra độ chính xác tập Validation
        print(f"Val accuracy:   {(100.0*result_dict['val']):4.2f}%")
    # In ra độ chính xác tập Test
    print(f"Test accuracy:  {(100.0*result_dict['test']):4.2f}%")

In [ ]:
node_mlp_model, node_mlp_result = train_node_classifier(model_name="MLP",
                                                        dataset=cora_dataset,
                                                        c_hidden=16,
                                                        num_layers=2,
                                                        dp_rate=0.1)

print_results(node_mlp_result)

GPU available: True, used: True
I1113 19:12:52.401648 139969460983616 distributed.py:49] GPU available: True, used: True
TPU available: False, using: 0 TPU cores
I1113 19:12:52.403518 139969460983616 distributed.py:49] TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
I1113 19:12:52.404974 139969460983616 accelerator_connector.py:385] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Found pretrained model, loading...
Train accuracy: 96.43%
Val accuracy:   52.60%
Test accuracy:  60.60%


/home/phillip/anaconda3/envs/nlp1/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Mặc dù **MLP** có thể bị **overfit** trên **training dataset** do các **high-dimensional input features**, nó không hoạt động quá tốt trên **test set**.

Bây giờ chúng ta sẽ chạy code để huấn luyện lần lượt: MLP (để lấy điểm chuẩn) và sau đó là GCN (để xem có cải thiện không).

In [ ]:
# Huấn luyện mô hình MLP (Baseline)
node_mlp_model, node_mlp_result = train_node_classifier(model_name="MLP",
                                                        dataset=cora_dataset,
                                                        c_hidden=16,
                                                        num_layers=2,
                                                        dp_rate=0.1)

# In kết quả
print_results(node_mlp_result)

Hãy xem liệu chúng ta có thể đánh bại điểm số này với các **graph networks** của chúng ta hay không:

In [ ]:
node_gnn_model, node_gnn_result = train_node_classifier(model_name="GNN",
                                                        layer_name="GCN",
                                                        dataset=cora_dataset,
                                                        c_hidden=16,
                                                        num_layers=2,
                                                        dp_rate=0.1)
print_results(node_gnn_result)

GPU available: True, used: True
I1113 19:12:53.906714 139969460983616 distributed.py:49] GPU available: True, used: True
TPU available: False, using: 0 TPU cores
I1113 19:12:53.907762 139969460983616 distributed.py:49] TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
I1113 19:12:53.909639 139969460983616 accelerator_connector.py:385] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Found pretrained model, loading...
Train accuracy: 100.00%
Val accuracy:   77.20%
Test accuracy:  82.40%


Như chúng ta đã kỳ vọng, **GNN model** vượt trội hơn **MLP** với một khoảng cách khá lớn. Điều này cho thấy việc sử dụng **graph information** thực sự cải thiện các **predictions** của chúng ta và giúp chúng ta **generalizes** (khái quát hóa) tốt hơn.

Các **hyperparameters** trong **model** đã được chọn để tạo ra một mạng tương đối nhỏ. Điều này là do lớp đầu tiên với **input dimension** là 1433 có thể tương đối tốn kém (về mặt tính toán) để thực hiện đối với các **large graphs**.

Nhìn chung, **GNNs** có thể trở nên tương đối tốn kém đối với các **very big graphs**. Đây là lý do tại sao các **GNNs** như vậy hoặc là có **hidden size** nhỏ, hoặc sử dụng một **batching strategy** đặc biệt nơi chúng ta **sample** (lấy mẫu) một **connected subgraph** (đồ thị con có kết nối) của **graph** lớn ban đầu.

### Edge-level tasks: Link prediction

Trong một số ứng dụng, chúng ta có thể phải dự đoán ở **edge-level** thay vì **node-level**. **Edge-level task** phổ biến nhất trong **GNN** là **link prediction**.

**Link prediction** có nghĩa là với một **graph** cho trước, chúng ta muốn dự đoán liệu sẽ có/nên có một **edge** giữa hai **nodes** hay không. Ví dụ, trong một **social network** (mạng xã hội), điều này được Facebook và các công ty tương tự sử dụng để đề xuất bạn mới cho bạn. Một lần nữa, **graph level information** có thể rất quan trọng để thực hiện **task** này.

**Output prediction** thường được thực hiện bằng cách tính toán một **similarity metric** (độ đo tương đồng) trên cặp **node features**, giá trị này nên là 1 nếu nên có một **link**, và ngược lại gần bằng 0.

Để giữ cho **tutorial** ngắn gọn, chúng ta sẽ không tự **implement** **task** này. Tuy nhiên, có rất nhiều tài nguyên tốt ngoài kia nếu bạn quan tâm đến việc xem xét kỹ hơn **task** này.

Các hướng dẫn và bài báo cho chủ đề này bao gồm:

* [PyTorch Geometric example](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/link_pred.py)
* [Graph Neural Networks: A Review of Methods and Applications](https://arxiv.org/pdf/1812.08434.pdf), Zhou et al. 2019
* [Link Prediction Based on Graph Neural Networks](https://papers.nips.cc/paper/2018/file/53f0d7c537d99b3824f0f99d62ea2428-Paper.pdf), Zhang and Chen, 2018.

### Graph-level tasks: Graph classification

Cuối cùng, trong phần này của **tutorial**, chúng ta sẽ xem xét kỹ hơn cách áp dụng **GNNs** vào **task** của **graph classification**.

Mục tiêu là phân loại một **entire graph** (toàn bộ đồ thị) thay vì các **single nodes** hay **edges**. Do đó, chúng ta cũng được cung cấp một **dataset** gồm **multiple graphs** mà chúng ta cần phân loại dựa trên một số **structural graph properties** (thuộc tính cấu trúc đồ thị).

**Task** phổ biến nhất cho **graph classification** là **molecular property prediction** (dự đoán tính chất phân tử), trong đó các **molecules** (phân tử) được biểu diễn dưới dạng **graphs**. Mỗi **atom** (nguyên tử) được liên kết với một **node**, và các **edges** trong **graph** là các **bonds** (liên kết hóa học) giữa các **atoms**. Ví dụ, hãy nhìn vào hình bên dưới.

<center width="100%"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/molecule_graph.svg?raw=1" width="600px"></center>

Ở bên trái, chúng ta có một **molecule** nhỏ, bất kỳ với các **atoms** khác nhau, trong khi phần bên phải của hình ảnh hiển thị **graph representation**.

Các **atom types** được trừu tượng hóa thành **node features** (ví dụ: một **one-hot vector**), và các **bond types** khác nhau được sử dụng làm **edge features**. Để đơn giản, chúng ta sẽ bỏ qua các **edge attributes** trong **tutorial** này, nhưng bạn có thể đưa vào bằng cách sử dụng các phương pháp như [Relational Graph Convolution](https://arxiv.org/abs/1703.06103) cái mà sử dụng một **weight matrix** khác nhau cho mỗi **edge type**.

**Dataset** chúng ta sẽ sử dụng bên dưới được gọi là **MUTAG dataset**. Nó là một **benchmark** nhỏ phổ biến cho các thuật toán **graph classification**, và chứa 188 **graphs** với trung bình 18 **nodes** và 20 **edges** cho mỗi **graph**.

Các **graph nodes** có 7 **labels/atom types** khác nhau, và các **binary graph labels** đại diện cho "hiệu ứng đột biến gen của chúng trên một loại vi khuẩn gram âm cụ thể" (ý nghĩa cụ thể của các **labels** không quá quan trọng ở đây).

**Dataset** này là một phần của bộ sưu tập lớn các **graph classification datasets** khác nhau, được biết đến là [TUDatasets](https://chrsmrrs.github.io/datasets/), có thể truy cập trực tiếp thông qua `torch_geometric.datasets.TUDataset` ([tài liệu](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.TUDataset)) trong **PyTorch Geometric**. Chúng ta có thể tải **dataset** bên dưới.

In [ ]:
tu_dataset = torch_geometric.datasets.TUDataset(root=DATASET_PATH, name="MUTAG")

Hãy xem một số thống kê về bộ dữ liệu:

In [ ]:
print("Data object:", tu_dataset.data)
print("Length:", len(tu_dataset))
print(f"Average label: {tu_dataset.data.y.float().mean().item():4.2f}")

Data object: Data(edge_attr=[7442, 4], edge_index=[2, 7442], x=[3371, 7], y=[188])
Length: 188
Average label: 0.66


Dòng đầu tiên cho thấy cách **dataset** lưu trữ các **graphs** khác nhau.

Các **nodes**, **edges**, và **labels** của mỗi **graph** được **concatenated** (nối lại) thành một **tensor** duy nhất, và **dataset** lưu trữ các chỉ số (**indices**) để chia tách các **tensors** một cách tương ứng.

Độ dài của **dataset** là số lượng **graphs** chúng ta có, và "**average label**" biểu thị phần trăm của **graph** có **label** là 1. Miễn là phần trăm nằm trong khoảng 0.5, chúng ta có một **dataset** tương đối **balanced** (cân bằng). Việc các **graph datasets** bị **imbalanced** (mất cân bằng) xảy ra khá thường xuyên, do đó việc kiểm tra **class balance** luôn là một việc tốt nên làm.

Tiếp theo, chúng ta sẽ chia **dataset** của mình thành phần **training** và **test**. Lưu ý rằng chúng ta không sử dụng **validation set** lần này do kích thước nhỏ của **dataset**.

Do đó, **model** của chúng ta có thể bị **overfit** nhẹ (do không có validation để dừng sớm), nhưng chúng ta vẫn có được một ước lượng về hiệu suất trên dữ liệu chưa được huấn luyện (**untrained data**).

In [ ]:
torch.manual_seed(42)
tu_dataset.shuffle()
train_dataset = tu_dataset[:150]
test_dataset = tu_dataset[150:]

Khi sử dụng một **data loader**, chúng ta gặp phải một vấn đề với việc **batching** $N$ **graphs**.

Mỗi **graph** trong **batch** có thể có số lượng **nodes** và **edges** khác nhau, và do đó chúng ta sẽ cần rất nhiều **padding** để thu được một **tensor** duy nhất.

**Torch geometric** sử dụng một cách tiếp cận khác, hiệu quả hơn: chúng ta có thể xem $N$ **graphs** trong một **batch** như là một **single large graph** (một đồ thị lớn duy nhất) với **concatenated node and edge list** (danh sách nút và cạnh được nối lại với nhau).

Vì không có **edge** nào giữa $N$ **graphs**, việc chạy các **GNN layers** trên **large graph** mang lại cho chúng ta **output** giống y hệt như việc chạy **GNN** trên từng **graph** riêng biệt. Về mặt hình ảnh, chiến lược **batching** này được trực quan hóa bên dưới (nguồn hình ảnh - nhóm PyTorch Geometric, [hướng dẫn tại đây](https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing#scrollTo=2owRWKcuoALo)).

<center width="100%"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/torch_geometric_stacking_graphs.png?raw=1" width="600px"></center>

**Adjacency matrix** sẽ bằng 0 đối với bất kỳ **nodes** nào đến từ hai **graphs** khác nhau, và ngược lại thì tuân theo **adjacency matrix** của **individual graph** (đồ thị cá nhân).

May mắn thay, chiến lược này đã được **implemented** sẵn trong **torch geometric**, và do đó chúng ta có thể sử dụng **data loader** tương ứng:

In [ ]:
# Tạo DataLoader cho tập huấn luyện (Training Set)
# batch_size=64: Mỗi lần đưa vào model 64 đồ thị phân tử cùng lúc
# shuffle=True: Xáo trộn dữ liệu mỗi epoch để model học tốt hơn, không bị học vẹt theo thứ tự
graph_train_loader = geom_data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Tạo DataLoader cho tập Validation (Sử dụng tạm test_dataset vì dữ liệu MUTAG quá nhỏ)
graph_val_loader = geom_data.DataLoader(test_dataset, batch_size=64)

# Tạo DataLoader cho tập Kiểm thử (Test Set)
graph_test_loader = geom_data.DataLoader(test_dataset, batch_size=64)

Hãy tải một batch bên dưới để xem cách hoạt động của việc batching:

In [ ]:
# Lấy ra batch đầu tiên từ test loader để kiểm tra
batch = next(iter(graph_test_loader))

print("Batch:", batch)
# In thử 10 nhãn đầu tiên (tương ứng với 10 đồ thị đầu trong batch)
print("Labels:", batch.y[:10])
# In thử 40 chỉ số batch đầu tiên (để xem các node thuộc về đồ thị nào)
print("Batch indices:", batch.batch[:40])

Batch: Batch(batch=[687], edge_attr=[1512, 4], edge_index=[2, 1512], x=[687, 7], y=[38])
Labels: tensor([1, 1, 1, 0, 0, 0, 1, 1, 1, 0])
Batch indices: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2])


Chúng ta có 38 **graphs** được xếp chồng lên nhau (**stacked together**) cho **test dataset**.

Các **batch indices**, được lưu trong `batch`, chỉ ra rằng 12 **nodes** đầu tiên thuộc về **graph** thứ nhất, 22 **nodes** tiếp theo thuộc về **graph** thứ hai, và cứ thế tiếp tục. Các **indices** này rất quan trọng để thực hiện **final prediction** (dự đoán cuối cùng).

Để thực hiện một **prediction** trên toàn bộ một **graph**, chúng ta thường thực hiện một **pooling operation** trên tất cả các **nodes** sau khi chạy **GNN model**. Trong trường hợp này, chúng ta sẽ sử dụng **average pooling** (gom nhóm trung bình).

Do đó, chúng ta cần biết những **nodes** nào nên được bao gồm trong **average pool** nào. Sử dụng **pooling** này, chúng ta đã có thể tạo **graph network** của mình bên dưới. Cụ thể, chúng ta tái sử dụng **class** `GNNModel` từ trước, và đơn giản là thêm một **average pool** và một **single linear layer** cho **graph prediction task**.

In [ ]:
class GraphGNNModel(nn.Module):

    def __init__(self, c_in, c_hidden, c_out, dp_rate_linear=0.5, **kwargs):
        """
        Inputs:
            c_in - Dimension của input features
            c_hidden - Dimension của hidden features
            c_out - Dimension của output features (thường là số lượng classes)
            dp_rate_linear - Dropout rate trước lớp linear (thường cao hơn nhiều so với bên trong GNN)
            kwargs - Các tham số bổ sung cho đối tượng GNNModel
        """
        super().__init__()
        # Khởi tạo phần GNN (Backbone) để trích xuất đặc trưng cho từng Node
        # Lưu ý: c_out của GNN lúc này là c_hidden, chưa phải là output cuối cùng!
        self.GNN = GNNModel(c_in=c_in,
                            c_hidden=c_hidden,
                            c_out=c_hidden,
                            **kwargs)

        # Phần "Head" (Đầu ra) dùng để phân loại sau khi đã Pooling
        self.head = nn.Sequential(
            nn.Dropout(dp_rate_linear),
            nn.Linear(c_hidden, c_out)
        )

    def forward(self, x, edge_index, batch_idx):
        """
        Inputs:
            x - Input features cho mỗi node
            edge_index - List các cặp chỉ số vertex biểu diễn edges (ký hiệu của PyTorch Geometric)
            batch_idx - Index của phần tử batch cho mỗi node (cho biết node thuộc về graph nào)
        """
        # Bước 1: Trích xuất đặc trưng cho từng node
        # Output: [Total Nodes, c_hidden]
        x = self.GNN(x, edge_index)

        # Bước 2: Pooling (Gom nhóm)
        # Biến đổi từ Node Embeddings -> Graph Embeddings
        # Lấy trung bình cộng các node thuộc cùng 1 graph (dựa vào batch_idx)
        # Output: [Batch Size, c_hidden]
        x = geom_nn.global_mean_pool(x, batch_idx)

        # Bước 3: Phân loại (Prediction)
        # Output: [Batch Size, c_out]
        x = self.head(x)
        return x

Cuối cùng, chúng ta có thể tạo một **PyTorch Lightning module** để xử lý việc **training**.

Nó tương tự như các **modules** chúng ta đã thấy trước đây và không làm gì đáng ngạc nhiên về mặt **training**. Vì chúng ta có một **binary classification task** (bài toán phân loại nhị phân), chúng ta sử dụng **Binary Cross Entropy loss**.

In [ ]:
class GraphLevelGNN(pl.LightningModule):

    def __init__(self, **model_kwargs):
        super().__init__()
        # Lưu hyperparameters
        self.save_hyperparameters()

        # Khởi tạo model GraphGNNModel (đã định nghĩa ở bước trước)
        self.model = GraphGNNModel(**model_kwargs)

        # Chọn hàm Loss function:
        # Nếu c_out (số lớp đầu ra) là 1 -> Bài toán Binary Classification -> Dùng BCEWithLogitsLoss
        # Nếu c_out > 1 -> Bài toán Multi-class Classification -> Dùng CrossEntropyLoss
        self.loss_module = nn.BCEWithLogitsLoss() if self.hparams.c_out == 1 else nn.CrossEntropyLoss()

    def forward(self, data, mode="train"):
        # Lấy dữ liệu từ batch
        # data.batch chính là batch_idx (vector chỉ định node nào thuộc graph nào)
        x, edge_index, batch_idx = data.x, data.edge_index, data.batch

        # Forward pass qua model
        x = self.model(x, edge_index, batch_idx)

        # Loại bỏ chiều cuối nếu kích thước là [Batch Size, 1] -> [Batch Size]
        x = x.squeeze(dim=-1)

        # Tính toán dự đoán (Predictions)
        if self.hparams.c_out == 1:
            # Đối với Binary: Giá trị > 0 tương ứng với xác suất sigmoid > 0.5 -> Class 1
            preds = (x > 0).float()
            # Chuyển label sang dạng float để tính BCE Loss
            data.y = data.y.float()
        else:
            # Đối với Multi-class: Lấy chỉ số có giá trị lớn nhất
            preds = x.argmax(dim=-1)

        # Tính Loss và Accuracy
        loss = self.loss_module(x, data.y)
        acc = (preds == data.y).sum().float() / preds.shape[0]
        return loss, acc

    def configure_optimizers(self):
        # Sử dụng AdamW optimizer
        # Learning rate cao (1e-2) vì dataset nhỏ và model nhỏ
        optimizer = optim.AdamW(self.parameters(), lr=1e-2, weight_decay=0.0)
        return optimizer

    def training_step(self, batch, batch_idx):
        loss, acc = self.forward(batch, mode="train")
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        return loss

    def validation_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="val")
        self.log('val_acc', acc)

    def test_step(self, batch, batch_idx):
        _, acc = self.forward(batch, mode="test")
        self.log('test_acc', acc)

Bên dưới, chúng ta huấn luyện **model** trên **dataset** của chúng ta. Nó giống với các **training functions** điển hình mà chúng ta đã thấy cho đến nay.

In [ ]:
def train_graph_classifier(model_name, **model_kwargs):
    pl.seed_everything(42)

    # Tạo một PyTorch Lightning trainer với callback để lưu checkpoint
    root_dir = os.path.join(CHECKPOINT_PATH, "GraphLevel" + model_name)
    os.makedirs(root_dir, exist_ok=True)

    trainer = pl.Trainer(default_root_dir=root_dir,
                         # ModelCheckpoint: Lưu lại trọng số tốt nhất dựa trên val_acc
                         callbacks=[ModelCheckpoint(save_weights_only=True, mode="max", monitor="val_acc")],
                         accelerator="gpu" if str(device).startswith("cuda") else "cpu",
                         devices=1,
                         max_epochs=500, # Train lâu hơn (500 epochs) vì dataset nhỏ và cần hội tụ kỹ
                         enable_progress_bar=False)

    trainer.logger._default_hp_metric = None # Tham số logging tùy chọn mà chúng ta không cần

    # Kiểm tra xem pretrained model đã tồn tại chưa. Nếu có, load nó và bỏ qua training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"GraphLevel{model_name}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Tìm thấy pretrained model, đang loading...")
        model = GraphLevelGNN.load_from_checkpoint(pretrained_filename)
    else:
        pl.seed_everything(42)
        # Khởi tạo model GraphLevelGNN
        # Logic c_out: Nếu bài toán có 2 class -> Output 1 node (Binary). Nếu > 2 class -> Output N node (Multi-class)
        model = GraphLevelGNN(c_in=tu_dataset.num_node_features,
                              c_out=1 if tu_dataset.num_classes==2 else tu_dataset.num_classes,
                              **model_kwargs)

        # Bắt đầu huấn luyện với train_loader và val_loader
        trainer.fit(model, graph_train_loader, graph_val_loader)
        # Load lại model tốt nhất sau khi train xong
        model = GraphLevelGNN.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)

    # Test model tốt nhất trên validation và test set
    # Lưu ý: PyTorch Lightning gọi hàm test nhưng trả về dictionary kết quả, ta lấy 'test_acc' từ đó
    train_result = trainer.test(model, graph_train_loader, verbose=False)
    test_result = trainer.test(model, graph_test_loader, verbose=False)

    result = {"test": test_result[0]['test_acc'], "train": train_result[0]['test_acc']}
    return model, result

Cuối cùng, hãy thực hiện việc **training** và **testing**. Hãy thoải mái thử nghiệm với các **GNN layers**, **hyperparameters** khác nhau, v.v.

In [ ]:
# Huấn luyện mô hình
# model_name: Tên dùng để lưu checkpoint
# c_hidden=256: Số lượng features ẩn lớn hơn nhiều so với bài toán Node-level (chỉ 16)
# layer_name="GraphConv": Sử dụng lớp GraphConv
model, result = train_graph_classifier(model_name="GraphConv",
                                       c_hidden=256,
                                       layer_name="GraphConv",
                                       num_layers=3,
                                       dp_rate_linear=0.5,
                                       dp_rate=0.0)

# In kết quả cuối cùng
print(f"GraphConv Results:")
print_results(result)

GPU available: True, used: True
I1113 19:12:54.045717 139969460983616 distributed.py:49] GPU available: True, used: True
TPU available: False, using: 0 TPU cores
I1113 19:12:54.047091 139969460983616 distributed.py:49] TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
I1113 19:12:54.048336 139969460983616 accelerator_connector.py:385] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/phillip/anaconda3/envs/nlp1/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


Found pretrained model, loading...


In [ ]:
# In ra kết quả hiệu suất của mô hình trên tập Train và tập Test.
print(f"Train performance: {100.0*result['train']:4.2f}%")
print(f"Test performance:  {100.0*result['test']:4.2f}%")

Train performance: 94.27%
Test performance:  92.11%


**Test performance** cho thấy rằng chúng ta đạt được điểm số khá tốt trên một phần **unseen** (chưa từng thấy) của **dataset**.

Cần lưu ý rằng vì chúng ta đã sử dụng **test set** cho cả **validation**, chúng ta có thể đã **overfitted** nhẹ vào tập này.

Tuy nhiên, **experiment** cho thấy rằng **GNNs** thực sự mạnh mẽ để dự đoán các tính chất của **graphs** và/hoặc **molecules**.

## Conclusion

Trong **tutorial** này, chúng ta đã thấy ứng dụng của **neural networks** vào các **graph structures**.

Chúng ta đã xem xét cách một **graph** có thể được biểu diễn (**adjacency matrix** hoặc **edge list**), và thảo luận về việc **implementation** các **graph layers** phổ biến: **GCN** và **GAT**. Các **implementations** đã cho thấy khía cạnh thực tế của các **layers**, điều mà thường dễ dàng hơn so với lý thuyết.

Cuối cùng, chúng ta đã thực nghiệm với các **tasks** khác nhau, ở các cấp độ **node-**, **edge-** và **graph-level**.

Nhìn chung, chúng ta đã thấy rằng việc bao gồm **graph information** trong các **predictions** có thể là cốt yếu để đạt được **high performance**. Có rất nhiều ứng dụng hưởng lợi từ **GNNs**, và tầm quan trọng của các mạng này có thể sẽ tăng lên trong những năm tới.